In [ ]:
##DATA PREPROCESSING
##NECESSARY LIBRARIES

import pandas as pd  ## loading data  like CSV files
import numpy as np ## numerical operations
import re, nltk  ## Regular expressions are used for pattern matching

from nltk.corpus import stopwords  ## stopwords are filtered out as they do not conatin any emotion stopwords are( the , is ,in)
from nltk.tokenize import word_tokenize  ## text into individual words
from nltk.stem import WordNetLemmatizer  ## to reduce words to their base just like for running it will be run
from sklearn.preprocessing import LabelEncoder ##encode categorical labels into numerical l abels
from sklearn.metrics import accuracy_score, classification_report ##encode categorical labels into numerical labels


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

##removes no not and nor from the set as they carry sentiment meaning
stop_words = set(stopwords.words('english'))
stop_words.remove("no")
stop_words.remove("not")
stop_words.remove("nor")
stop_words = [x.lower() for x in stop_words] ##all stopwords are converted to lowercase


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
## this both libraries are used for handling the imbalaneced data and to check performance and efficiency of algorithms

!pip install lightgbm
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 4.1 MB/s eta 0:00:00


In [ ]:
## PREDEFINED WORDS

contractions = {
        "ai\snot": "am not",
        "wo\snot": "will not",
        "would\snot": "would not",
        "should\snot": "should not",
        "isn\snot": "is not",
        "aren\snot": "are not",
        "wasn\snot": "was not",
        "weren\snot": "were not",
        "haven\snot": "have not",
        "hasn\snot": "has not",
        "hadn\snot": "had not",
        "don\snot": "do not",
        "doesn\snot": "does not",
        "didn\snot": "did not",
        "can\snot": "can not",
        "cannot": "can not",
        "couldn\snot": "could not",
        "shouldn\snot": "should not",
        "mightn\snot": "might not",
        "mustn\snot": "must not",
        "shan\snot": "shall not",
        "won\snot": "will not",
        "ain\snot": "am not",
        "I\sam": "I am",
        "you\sare": "you are",
        "he\sis": "he is",
        "she\sis": "she is",
        "it\sis": "it is",
        "we\sare": "we are",
        "they\sare": "they are",
        "I\swould": "I would",
        "you\swould": "you would",
        "he\swould": "he would",
        "she\swould": "she would",
        "it\swould": "it would",
        "we\swould": "we would",
        "they\swould": "they would",
        "I\scould": "I could",
        "you\scould": "you could",
        "he\scould": "he could",
        "she\scould": "she could",
        "it\scould": "it could",
        "we\scould": "we could",
        "they\scould": "they could",
        "I\shave": "I have",
        "you\shave": "you have",
        "he\shas": "he has",
        "she\shas": "she has",
        "it\shas": "it has",
        "we\shave": "we have",
        "they\shave": "they have",
        "I\swill": "I will",
        "you\swill": "you will",
        "he\swill": "he will",
        "she\swill": "she will",
        "it\swill": "it will",
        "we\swill": "we will",
        "they\swill": "they will",
        "I\smust": "I must",
        "you\smust": "you must",
        "he\smust": "he must",
        "she\smust": "she must",
        "it\smust": "it must",
        "we\smust": "we must",
        "they\smust": "they must",
        "I\sshall": "I shall",
        "you\sshall": "you shall",
        "he\sshall": "he shall",
        "she\sshall": "she shall",
        "it\sshall": "it shall",
        "we\sshall": "we shall",
        "they\sshall": "they shall",
        "haven\s't": "have not",
        "hasn\s't": "has not",
        "hadn\s't": "had not",
        "don\s't": "do not",
        "doesn\s't": "does not",
        "didn\s't": "did not",
        "can\s't": "can not",
        "cannot": "can not",
        "couldn\s't": "could not",
        "shouldn\s't": "should not",
        "mightn\s't": "might not",
        "mustn\s't": "must not",
        "shan\s't": "shall not",
        "won\s't": "will not",
        "ain\s't": "am not",
        "aren\s't": "are not",
        "wasn\s't": "was not",
        "weren\s't": "were not",
        "I\sdidn't": "I did not",
        "you\sdidn't": "you did not",
        "he\sdidn't": "he did not",
        "she\sdidn't": "she did not",
        "it\sdidn't": "it did not",
        "we\sdidn't": "we did not",
        "they\sdidn't": "they did not",
        "I\scannot": "I can not",
        "you\scannot": "you can not",
        "he\scannot": "he can not",
        "she\scannot": "she can not",
        "it\scannot": "it can not",
        "we\scannot": "we can not",
        "they\scannot": "they can not",
        "I\swon't": "I will not",
        "you\swon't": "you will not",
        "he\swon't": "he will not",
        "she\swon't": "she will not",
        "it\swon't": "it will not",
        "we\swon't": "we will not",
        "they\swon't": "they will not",
        "I\shasn't": "I has not",
        "you\shasn't": "you has not",
        "he\shasn't": "he has not",
        "she\shasn't": "she has not",
        "it\shasn't": "it has not",
        "we\shasn't": "we has not",
        "they\shasn't": "they has not"
}

nots = {
    'not sad': 'Happy', 'not bad': 'Happy', 'not boring': 'Happy', 'not wrong': 'Happy', 'not bored': 'Happy',
        'not jealous': 'Happy', 'not happy': 'Sad', 'not well': 'Sad', 'not suitable': 'Angry',
        'not right': 'Angry', 'not good': 'Sad', 'not excited': 'Angry', 'not funny ': 'Sad', 'not kind': 'Sad',
        'not proud': 'Angry', 'not cool': 'Angry', 'not funny': 'Angry', 'not kind': 'Angry', 'not open': 'Angry',
        'not safe': 'Fear', 'not enough': 'Empty', 'not know': 'Sad', 'not knowing': 'Sad', 'not believe': 'Angry',
        'not believing': 'Angry', 'not understand': 'Sad', 'not understanding': 'Sad', 'no doubt': 'Happy',
        'not think': 'Sad', 'not thinking': 'Sad', 'not recognise': 'Sad', 'not recognising': 'Sad',
        'not forget': 'Angry', 'not forgetting': 'Angry', 'not remember': 'Sad', 'not remembering': 'Sad',
        'not imagine': 'Sad', 'not imagining': 'Sad', 'not mean': 'Sad', 'not meaning': 'Sad',
        'not agree': 'Angry', 'not agreeing': 'Sad', 'not disagree': 'Happy', 'not disagreeing': 'Happy',
        'not deny': 'Sad', 'not denying': 'Sad', 'not promise': 'Angry', 'not promising': 'Angry',
        'not satisfy': 'Sad', 'not satisfying': 'Sad', 'not realise': 'Sad', 'not realising': 'Sad',
        'not appear': 'Angry', 'not appearing': 'Angry', 'not please': 'Sad', 'not pleasing': 'Sad',
        'not impress': 'Sad', 'not impressing': 'Sad', 'not surprise': 'Sad', 'not surprising': 'Sad',
        'not concern': 'Sad', 'not concerning': 'Sad', 'not have': 'Sad', 'not having': 'Sad',
        'not own': 'Sad', 'not owning': 'Sad', 'not possess': 'Sad', 'not possessing': 'Sad',
        'not lack': 'Sad', 'not lacking': 'Sad', 'not consist': 'Sad', 'not consisting': 'Sad',
        'not involve': 'Sad', 'not involving': 'Sad', 'not include': 'Sad', 'not including': 'Sad',
        'not contain': 'Sad', 'not containing': 'Sad', 'not love': 'Sad', 'not like': 'Angry',
        'not hate': 'Happy', 'not hating': 'Happy', 'not adore': 'Sad', 'not adoring': 'Sad',
        'not prefer': 'Sad', 'not preferring': 'Sad', 'not care': 'Angry', 'not mind': 'Angry',
        'not minding': 'Sad', 'not want': 'Angry', 'not wanting': 'Sad', 'not need': 'Angry',
        'not needing': 'Angry', 'not desire': 'Sad', 'not desiring': 'Sad', 'not wish': 'Sad',
        'not wishing': 'Sad', 'not hope': 'Sad', 'not hoping': 'Sad', 'not appreciate': 'Sad',
        'not appreciating': 'Sad', 'not value': 'Sad', 'not valuing': 'Sad', 'not owe': 'Sad',
        'not owing': 'Sad', 'not seem': 'Sad', 'not seeming': 'Sad', 'not fit': 'Sad', 'not fitting': 'Sad',
        'not depend': 'Sad', 'not depending': 'Sad', 'not matter': 'Sad', 'not afford': 'Sad',
        'not affording': 'Sad', 'not aim': 'Sad', 'not aiming': 'Sad', 'not attempt': 'Angry',
        'not attempting': 'Angry', 'not ask': 'Angry', 'not asking': 'Angry', 'not arrange': 'Angry',
        'not arranging': 'Angry', 'not beg': 'Angry', 'not begging': 'Angry', 'not begin': 'Angry',
        'not beginning': 'Angry', 'not caring': 'Angry', 'not choose': 'Angry', 'not choosing': 'Angry',
        'not claim': 'Angry', 'not claiming': 'Angry', 'not consent': 'Angry', 'not consenting': 'Angry',
        'not continue': 'Angry', 'not continuing': 'Angry', 'not dare': 'Angry', 'not daring': 'Angry',
        'not decide': 'Sad', 'not deciding': 'Sad', 'not demand': 'Angry', 'not demanding': 'Angry',
        'not deserve': 'Angry', 'not deserving': 'Angry', 'not expect': 'Angry', 'not expecting': 'Angry',
        'not fail': 'Happy', 'not failing': 'Happy', 'not get': 'Sad', 'not getting': 'Sad',
        'not hesitate': 'Sad', 'not hesitating': 'Sad', 'not hurry': 'Happy', 'not hurrying': 'Happy',
        'not intend': 'Sad', 'not intending': 'Sad', 'not learn': 'Angry', 'not learning': 'Angry',
        'not liking': 'Angry', 'not loving': 'Sad', 'not manage': 'Angry', 'not managing': 'Angry',
        'not neglect': 'Sad', 'not neglecting': 'Sad', 'not offer': 'Angry', 'not offering': 'Angry',
        'not plan': 'Angry', 'not planing': 'Angry', 'not prepare': 'Angry', 'not preparing': 'Angry',
        'not pretend': 'Angry', 'not pretending': 'Angry', 'not proceed': 'Angry', 'not proceeding': 'Angry',
        'not propose': 'Angry', 'not proposing': 'Sad', 'not refuse': 'Sad', 'not refusing': 'Sad',
        'not start': 'Sad', 'not starting': 'Sad', 'not stop': 'Happy', 'not stopping': 'Happy',
        'not struggle': 'Angry', 'not struggling': 'Angry', 'not swear': 'Angry', 'not swearing': 'Angry',
        'not threaten': 'Happy', 'not threatening': 'Happy', 'not try': 'Angry', 'not trying': 'Angry',
        'not volunteer': 'Angry', 'not volunteering': 'Angry', 'not wait': 'Angry', 'not waiting': 'Angry',
        'not feel': 'Sad', 'not feeling': 'Sad', "not able": "Sad", "not do": "Sad",
        'not apologize': 'Sad', 'not apologizing': 'Sad', 'not forgive': 'Angry', 'not forgiving': 'Angry',
        'not trust': 'Angry', 'not trusting': 'Angry', 'not regret': 'Angry', 'not regretting': 'Angry',
        'not rejoice': 'Sad', 'not rejoicing': 'Sad', 'not admire': 'Sad', 'not admiring': 'Sad',
        'not compliment': 'Sad', 'not complimenting': 'Sad', 'not criticize': 'Happy', 'not criticizing': 'Happy',
        'not encourage': 'Angry', 'not encouraging': 'Angry', 'not insult': 'Sad', 'not insulting': 'Sad',
        'not praise': 'Angry', 'not praising': 'Angry', 'not support': 'Angry', 'not supporting': 'Angry',
        'not blame': 'Sad', 'not blaming': 'Sad', 'not defend': 'Sad', 'not defending': 'Sad',
        'not appreciate': 'Sad', 'not appreciating': 'Sad', 'not enjoy': 'Sad', 'not enjoying': 'Sad',
        'not like': 'Angry', 'not liking': 'Angry', 'not love': 'Sad', 'not loving': 'Sad',
        'not prefer': 'Sad', 'not preferring': 'Sad', 'not want': 'Angry', 'not wanting': 'Sad',
        'not believe': 'Angry', 'not believing': 'Angry', 'not doubt': 'Happy', 'not doubting': 'Happy',
        'not imagine': 'Sad', 'not imagining': 'Sad', 'not realize': 'Sad', 'not realizing': 'Sad',
        'not remember': 'Sad', 'not remembering': 'Sad', 'not recognize': 'Sad', 'not recognizing': 'Sad',
        'not consider': 'Sad', 'not considering': 'Sad', 'not think': 'Sad', 'not thinking': 'Sad',
        'not forget': 'Angry', 'not forgetting': 'Angry', 'not ignore': 'Angry', 'not ignoring': 'Angry',
        'not overlook': 'Angry', 'not overlooking': 'Angry', 'not understand': 'Sad', 'not understanding': 'Sad',
        'not hear': 'Angry', 'not hearing': 'Angry', 'not listen': 'Angry', 'not listening': 'Angry',
        'not look': 'Angry', 'not looking': 'Angry', 'not smell': 'Angry', 'not smelling': 'Angry',
        'not taste': 'Angry', 'not tasting': 'Angry', 'not touch': 'Angry', 'not touching': 'Angry',
        'not feel': 'Sad', 'not feeling': 'Sad', 'not sense': 'Sad', 'not sensing': 'Sad',
        'not suppose': 'Angry', 'not supposing': 'Angry', 'not expect': 'Angry', 'not expecting': 'Angry',
        'not wait': 'Angry', 'not waiting': 'Angry', 'not long': 'Angry', 'not longing': 'Angry',
        'not yearn': 'Angry', 'not yearning': 'Angry', 'not wish': 'Sad', 'not wishing': 'Sad',
        'not hope': 'Sad', 'not hoping': 'Sad', 'not desire': 'Sad', 'not desiring': 'Sad',
        'not miss': 'Angry', 'not missing': 'Angry', 'not need': 'Angry', 'not needing': 'Angry',
        'not want': 'Angry', 'not wanting': 'Sad', 'not require': 'Angry', 'not requiring': 'Angry',
        'not demand': 'Angry', 'not demanding': 'Angry', 'not insist': 'Angry', 'not insisting': 'Angry',
        'not force': 'Angry', 'not forcing': 'Angry', 'not push': 'Angry', 'not pushing': 'Angry',
        'not pull': 'Angry', 'not pulling': 'Angry', 'not drag': 'Angry', 'not dragging': 'Angry',
        'not carry': 'Angry', 'not carrying': 'Angry', 'not lift': 'Angry', 'not lifting': 'Angry',
        'not drop': 'Angry', 'not dropping': 'Angry', 'not throw': 'Angry', 'not throwing': 'Angry',
        'not catch': 'Angry', 'not catching': 'Angry', 'not capture': 'Angry', 'not capturing': 'Angry',
        'not grab': 'Angry', 'not grabbing': 'Angry', 'not touch': 'Angry', 'not touching': 'Angry',
        'not reach': 'Angry', 'not reaching': 'Angry', 'not approach': 'Angry', 'not approaching': 'Angry',
        'not avoid': 'Angry', 'not avoiding': 'Angry', 'not evade': 'Angry', 'not evading': 'Angry',
        'not elude': 'Angry', 'not eluding': 'Angry', 'not escape': 'Angry', 'not escaping': 'Angry',
        'not run': 'Angry', 'not running': 'Angry', 'not jog': 'Angry', 'not jogging': 'Angry',
        'not walk': 'Angry', 'not walking': 'Angry', 'not crawl': 'Angry', 'not crawling': 'Angry',
        'not sneak': 'Angry', 'not sneaking': 'Angry', 'not tiptoe': 'Angry', 'not tiptoeing': 'Angry',
        'not dance': 'Angry', 'not dancing': 'Angry', 'not stomp': 'Angry', 'not stomping': 'Angry',
        'not shake': 'Angry', 'not shaking': 'Angry', 'not tremble': 'Angry', 'not trembling': 'Angry',
        'not shiver': 'Angry', 'not shivering': 'Angry', 'not quiver': 'Angry', 'not quivering': 'Angry',
        'not vibrate': 'Angry', 'not vibrating': 'Angry', 'not pulsate': 'Angry', 'not pulsating': 'Angry',
        'not throb': 'Angry', 'not throbbing': 'Angry', 'not beat': 'Angry', 'not beating': 'Angry',
        'not palpitate': 'Angry', 'not palpitating': 'Angry', 'not pump': 'Angry', 'not pumping': 'Angry',
        'not glide': 'Angry', 'not gliding': 'Angry', 'not slide': 'Angry', 'not sliding': 'Angry',
        'not slip': 'Angry', 'not slipping': 'Angry', 'not skid': 'Angry'
}

shortcuts = {
    'u': 'you', 'y': 'why', 'r': 'are', 'doin': 'doing', 'hw': 'how', 'k': 'okay', 'm': 'am',
    'b4': 'before',
                   'idc': "i do not care", 'ty': 'thank you', 'wlcm': 'welcome', 'bc': 'because', '<3': 'love',
                   'xoxo': 'love',
                   'ttyl': 'talk to you later', 'gr8': 'great', 'bday': 'birthday', 'awsm': 'awesome', 'gud': 'good',
                   'h8': 'hate',
                   'lv': 'love', 'dm': 'direct message', 'rt': 'retweet', 'wtf': 'hate', 'idgaf': 'hate',
                   'irl': 'in real life', 'yolo': 'you only live once', "don't": "do not", 'g8': 'great',
                   "won't": "will not", 'tbh': 'to be honest', 'caj': 'casual', 'Ikr': 'I know, right?',
                   'omw': 'on my way',
                   'ofc': 'of course', 'Idc': "I don't care", 'Irl': 'In real life', 'tbf': 'To be fair',
                   'obvs': 'obviously', 'v': 'very', 'atm': 'at the moment',
                   'col': 'crying out loud', 'gbu': 'god bless you', 'gby': 'god bless you', 'gotcha': 'I got you',
                   'hehe': 'laughing', 'haha': 'laughing', 'hf': 'have fun',
                   'hry': 'hurry', 'hw': 'hardwork', 'idc': 'i don’t care', 'ikr': 'i know right', 'k': 'ok',
                   'lmao': 'laughing my ass off', 'lol': 'laughing out loud',
                   'n1': 'nice one', 'na': 'not available', 'qt': 'cutie', 'qtpi': 'cutie pie',
                   'rip': 'rest in peace',
                   'sry': 'sorry', 'tc': 'take care',
                   'thnks': 'thanks', 'thx': 'thanks', 'thnk': 'thanks', 'ttyl': 'talk to you later', 'txt': 'text',
                   'ugh': 'disgusted', 'w8': 'wait', "not sad": "happy"
}


In [ ]:
##WE ARE IMPORTING THE DATASET

dataset = pd.read_csv("/content/text2emotion.csv", header = None)
X = dataset[0].values ##selects the first column of the dataset and it has text data
y = dataset[1].values ##selects the second column of the dataset and it has sentiment(emotion)

In [ ]:
##HERE WE ARE LABELLING ALL THE EMOTIONS

from sklearn.preprocessing import LabelEncoder ## encode categorical labels into numerical label
l = LabelEncoder() ## perform label encoding
y = l.fit_transform(y) ##fits the encoder to the label(y) and then transforms them into numerical label
emotions = l.inverse_transform([0,1,2,3,4,5]) ##numerical labels back to categorical values
print(len(y))
print(emotions)

19999
['anger' 'fear' 'joy' 'love' 'sadness' 'surprise']


In [ ]:
##NOW WE WILL CLEAN THE TEXT

def remove_contradictions(text):
    if "n't" in text:
        text = text.replace("n't", " not") ##If n't is found then  it will replace it with not
    for pattern, replacement in contractions.items():
        text = re.sub(pattern, replacement, text)
    return text

In [ ]:
def removing_not(text):
        f = re.findall("not\s\w+", text) ## look for all the not in the sentence
        for i in f:
            try:
                text = text.replace(i, nots[i]) ##replace all the nots just like for not happy it will replace as sad
            except:
                pass
        text = text.lower()
        return text

In [ ]:
## FOLLOWING FUNCTION WILL REPLACE ALL THE SHORTCCUTS WITH IT ORIGINALL WORDS
def removing_shortcuts(text):
    full_words = []

    for token in text:
        if token in shortcuts.keys():
            token = shortcuts[token]
        full_words.append(token)
    text = " ".join(full_words)
    return text

In [ ]:
## IT WILL RETURN THE WORD IF NOT FOUND IN STOPWORDS
def removing_stopwords(text):
    return [word for word in text if not word in stop_words]

In [ ]:
## REMOVING WEBSITES LINKS STOPWORDS DIGITS
corpus = []
for i in X:
  review = re.sub("[^a-zA-Z]"," ",i)
  review = review.lower()
  review = re.sub(r'http\S+|www.\S+', '', review)
  review = remove_contradictions(review)
  review = removing_not(review)
  review = review.split()
  review = removing_shortcuts(review)
  review = ' '.join([i for i in review.split() if not i.isdigit()])
  review = word_tokenize(review)
  review = removing_stopwords(review)
  lemma = WordNetLemmatizer()
  review = [lemma.lemmatize(word) for word in review]
  review = " ".join(review)
  corpus.append(review)

In [ ]:
## CREATING BAG OF WORDS
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2000)
X = cv.fit_transform(corpus).toarray()
print(len(X),len(X[0]))

19999 2000


In [ ]:
## TRAINING TESTING
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


In [ ]:
## GB MODEL

from sklearn.ensemble import GradientBoostingClassifier
GB_classifier = GradientBoostingClassifier()
GB_classifier.fit(X_train, y_train)
y_pred = GB_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 84.6250%
Classification Report:
              precision    recall  f1-score   support

       anger       0.94      0.76      0.84       531
        fear       0.95      0.71      0.81       496
         joy       0.76      0.95      0.85      1372
        love       0.84      0.75      0.80       323
     sadness       0.94      0.85      0.89      1131
    surprise       0.68      0.85      0.75       147

    accuracy                           0.85      4000
   macro avg       0.85      0.81      0.82      4000
weighted avg       0.86      0.85      0.85      4000



In [ ]:
#CATBOOST MODEL

from catboost import CatBoostClassifier ## catboost may produce a lot of loging output by default thats why we have used logging level
CB_classifier= CatBoostClassifier(logging_level='Silent') ## setting logging level parameter to silent because we dont want to make excessive log files , so it will reduce the noise and train model faster
CB_classifier.fit(X_train, y_train)
y_pred = CB_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 89.8000%
Classification Report:
              precision    recall  f1-score   support

       anger       0.89      0.89      0.89       531
        fear       0.90      0.85      0.87       496
         joy       0.88      0.95      0.91      1372
        love       0.88      0.72      0.79       323
     sadness       0.95      0.93      0.94      1131
    surprise       0.76      0.73      0.75       147

    accuracy                           0.90      4000
   macro avg       0.88      0.85      0.86      4000
weighted avg       0.90      0.90      0.90      4000



In [ ]:
#LightGBM

from lightgbm import LGBMClassifier
LGBM_classifier = LGBMClassifier()
LGBM_classifier.fit(X_train, y_train)
y_pred = LGBM_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.208916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3175
[LightGBM] [Info] Number of data points in the train set: 15999, number of used features: 1190
[LightGBM] [Info] Start training from score -1.994119
[LightGBM] [Info] Start training from score -2.142851
[LightGBM] [Info] Start training from score -1.088352
[LightGBM] [Info] Start training from score -2.496411
[LightGBM] [Info] Start training from score -1.232224
[LightGBM] [Info] Start training from score -3.331143
Accuracy = 88.1250%
Classification Report:
              precision    recall  f1-score   support

       anger       0.90      0.85      0.87       531
        fear       0.87      0.82      0.85       496
         joy       0.87      0.92      0.89      1372
        love       0.77      0.84      0.80       323
     sa

In [ ]:
#decision tree model

from sklearn.tree import DecisionTreeClassifier
decision_classifier = DecisionTreeClassifier()
decision_classifier.fit(X_train, y_train)
y_pred = decision_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 83.5250%
Classification Report:
              precision    recall  f1-score   support

       anger       0.78      0.86      0.82       531
        fear       0.79      0.83      0.81       496
         joy       0.89      0.80      0.84      1372
        love       0.72      0.76      0.74       323
     sadness       0.88      0.90      0.89      1131
    surprise       0.66      0.77      0.71       147

    accuracy                           0.84      4000
   macro avg       0.79      0.82      0.80      4000
weighted avg       0.84      0.84      0.84      4000



In [ ]:
#kernal SVM

from sklearn.svm import SVC
kernal_classifier = SVC(kernel='rbf')
kernal_classifier.fit(X_train, y_train)
y_pred = kernal_classifier.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))

Accuracy = 84.3750%
Classification Report:
              precision    recall  f1-score   support

       anger       0.89      0.75      0.81       531
        fear       0.88      0.72      0.79       496
         joy       0.79      0.96      0.86      1372
        love       0.89      0.58      0.70       323
     sadness       0.88      0.92      0.90      1131
    surprise       0.93      0.54      0.68       147

    accuracy                           0.84      4000
   macro avg       0.88      0.74      0.79      4000
weighted avg       0.85      0.84      0.84      4000



In [ ]:
#VOTING CLASSIFIER
from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import GradientBoostingClassifier
GB_classifier = GradientBoostingClassifier()

from lightgbm import LGBMClassifier
LGBM_classifier = LGBMClassifier()

from sklearn.tree import DecisionTreeClassifier
decision_classifier = DecisionTreeClassifier()

from sklearn.svm import SVC
kernal_classifier = SVC(kernel = 'rbf')

##from catboost import CatBoostClassifier
##CB_classifier= CatBoostClassifier(logging_level='Silent')

ensemble = VotingClassifier(
    estimators=[
        ('GBC', GB_classifier),
        ('LGBMC', LGBM_classifier),
        ('DTC', decision_classifier),
        ('KSVMC', kernal_classifier)
       ## ('CBC',CB_classifier)
    ],
    voting='hard')##The voting parameter is set to 'hard', indicating that the final prediction of the ensemble will be based on a majority vote among the classifiers
ensemble.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.223091 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3175
[LightGBM] [Info] Number of data points in the train set: 15999, number of used features: 1190
[LightGBM] [Info] Start training from score -1.994119
[LightGBM] [Info] Start training from score -2.142851
[LightGBM] [Info] Start training from score -1.088352
[LightGBM] [Info] Start training from score -2.496411
[LightGBM] [Info] Start training from score -1.232224
[LightGBM] [Info] Start training from score -3.331143


VotingClassifier(estimators=[('GBC', GradientBoostingClassifier()),
                             ('LGBMC', LGBMClassifier()),
                             ('DTC', DecisionTreeClassifier()),
                             ('KSVMC', SVC())])

In [ ]:
y_pred = ensemble.predict(X_test)
print(f"Accuracy = {accuracy_score(y_test, y_pred)*100:2.4f}%")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=emotions))


Accuracy = 88.2250%
Classification Report:
              precision    recall  f1-score   support

       anger       0.89      0.85      0.87       531
        fear       0.89      0.84      0.86       496
         joy       0.84      0.96      0.90      1372
        love       0.89      0.70      0.78       323
     sadness       0.95      0.89      0.92      1131
    surprise       0.78      0.72      0.75       147

    accuracy                           0.88      4000
   macro avg       0.87      0.83      0.85      4000
weighted avg       0.89      0.88      0.88      4000



In [ ]:
##predict the result


def predict_emotion(text, classifier):
  text = text.replace("\n"," ")
  review = re.sub("[^a-zA-Z]"," ",text)## regular expression to remove any characters that are not alphabetic
  review = review.lower()
  review = re.sub(r'http\S+|www.\S+', '', review)
  review = remove_contradictions(review)
  review = removing_not(review)
  review = review.split()
  review = removing_shortcuts(review)
  review = ' '.join([i for i in review.split() if not i.isdigit()])##removes digits from the text
  review = word_tokenize(review)
  review = removing_stopwords(review)
  lemma = WordNetLemmatizer()
  review = [lemma.lemmatize(word) for word in review]
  review = " ".join(review)
  test = cv.transform([review]).toarray()
  result = classifier.predict(test)
  result = l.inverse_transform(result)
  return result

In [ ]:
text=''' Hrutuja was feeling low'''

print(predict_emotion(text, ensemble))

['sadness']


In [ ]:
text=''' hrutuja was not able to do connection so she is feeling low'''
print(predict_emotion(text, ensemble))

['sadness']


In [ ]:
text=''' ashwini was too excited as she has taken the autograph of M.S Dhoni'''
print(predict_emotion(text, ensemble))

['joy']
